In [1]:
import numpy as np
import time
import datetime
import numba
import math
import itertools


In [2]:
# lets just figure out if there is an immediate bias if we restrict to scenarios when perfect for adjacent pairs

maxvalue = 11
terms = 5



perms = np.array(list(itertools.product(range(maxvalue), repeat=terms)))
print(perms)
print(perms[71])
print(len(perms))
print(type(perms[0,0]))
print(len(perms)**2)


# # ab, ba, bc, cb, cd, dc, de, ed
# # 01, 10, 12, 21, 

# recplays = np.array([0, 1, 0, 1, 0, 1, 0, 1], dtype = np.int64)

# 01, 12, 23, 34, 10, 21, 32, 43
Arecplays = np.concatenate([np.zeros((terms-1), dtype = np.int64), np.ones((terms-1), dtype = np.int64)])
print(Arecplays)


left = perms[3112]
right = perms[2254]

print(left)
print(right)

bins = []

for idx000 in range(terms-1):
    bins.append([left[idx000], right[idx000+1]])
for idx000 in range(terms-1):
    bins.append([left[idx000+1], right[idx000]])
print(bins)
bins = np.array(bins)

testbins = [[left[1], right[3]], [right[3], left[1]]]
testbins = np.array(testbins)
print(testbins)
recans = [0, 1]

[[ 0  0  0  0  0]
 [ 0  0  0  0  1]
 [ 0  0  0  0  2]
 ...
 [10 10 10 10  8]
 [10 10 10 10  9]
 [10 10 10 10 10]]
[0 0 0 6 5]
161051
<class 'numpy.int64'>
25937424601
[0 0 0 0 1 1 1 1]
[ 0  2  3  7 10]
[ 0  1  7  6 10]
[[0, 1], [2, 7], [3, 6], [7, 10], [2, 0], [3, 1], [7, 7], [10, 6]]
[[2 6]
 [6 2]]


In [4]:
@numba.njit
def isin(a, b):
    out=np.zeros(len(a), dtype = numba.int64)
    for i in range(len(a)):
        for j in range(len(b)):
            if (a[i][0] == b[j][0]) & (a[i][1] == b[j][1]):
                out[i] = 1
#                 print('horray!')
    return out

@numba.njit
def contradiction(cbins, crplays):
    con = 0
    for ii in range(len(cbins)):
        for jj in range(len(cbins)):
            x = cbins[ii]
            y = cbins[jj]
            if (x[0] == y[0]) & (x[1] == y[1]):
                if crplays[ii] != crplays[jj]:
                    con = 1
    return con

@numba.njit
def duplicates(dbins):
    dup = 0
    for iii in range(len(dbins)):
        for jjj in range(len(dbins)):
            if iii != jjj:
                if (dbins[iii][0] == dbins[jjj][0]) & (dbins[iii][1] == dbins[jjj][1]):
                    dup = 1
    return dup

In [5]:
testout = isin(testbins, bins)
print(testout)

print(np.zeros((5,2)))

testcbins = np.array([[4, 3], [4, 0], [4, 0], [2, 4], [4, 3], [4, 3], [2, 0], [2, 0]])
testcrplays = np.array([0, 0, 0, 0, 1, 1, 1, 1])

test1 = contradiction(testcbins, testcrplays)
print(test1)

testcbins2 = np.array([[4, 0], [4, 0], [4, 0], [2, 4], [4, 3], [4, 3], [2, 0], [2, 0]])
testcrplays2 = np.array([0, 0, 0, 0, 1, 1, 1, 1])

test2 = contradiction(testcbins2, testcrplays2)
print(test2)

[0 0]
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
1
0


In [7]:
@numba.njit(parallel=True)
def full(prms, t, m, recplays):
    recans = [0, 1]
    count = 0
    count_con = 0
    count_train_success = 0
    count_tsuc_and_test_isin = 0
    count_tsuc_and_test_isout = 0
    count_test_isin_and_good = 0 
    count_test_isin_and_bad = 0 
    count_test_isin_and_neutral = 0
    count_test_isin_and_goodx2 = 0 
    count_test_isin_and_badx2 = 0 
    dup_count_test_isin_and_good = 0 
    dup_count_test_isin_and_bad = 0 
    dup_count_test_isin_and_neutral = 0
    dup_count_test_isin_and_goodx2 = 0 
    dup_count_test_isin_and_badx2 = 0 
    for ij000 in numba.prange(len(prms)):
        for ij001 in numba.prange(len(prms)):
            l = prms[ij000]
            r = prms[ij001]
            count += 1
            bns = np.zeros(((2*(t-1)), 2), dtype = numba.int64)
            for idx000 in range(t-1):
                bns[idx000][0] = l[idx000] 
                bns[idx000][1] = r[idx000+1]
            for idx001 in range(t-1):
                bns[t-1+idx001][0] = l[idx001+1] 
                bns[t-1+idx001][1] = r[idx001]
            if (ij000*100000+ij001)%1000000000 == 0:
                print(f'count={count} count_con={count_con} count_train_success={count_train_success}')
                print(f'count_tsuc_and_test_isin={count_tsuc_and_test_isin} count_tsuc_and_test_isout={count_tsuc_and_test_isout}')
                print(f'count_test_isin_and_good={count_test_isin_and_good} count_test_isin_and_bad={count_test_isin_and_bad} count_test_isin_and_neutral={count_test_isin_and_neutral}')
                print(f'count_test_isin_and_goodx2={count_test_isin_and_goodx2} count_test_isin_and_badx2={count_test_isin_and_badx2}')
                print('____________________________________________')
            # this finishes bns
            if contradiction(bns, recplays) == 1:
                count_con += 1
            else:
                count_train_success +=1
                testbns = [[l[1], r[3]], [r[3], l[1]]]
                check_isin = isin(testbns, bns)
                dp = duplicates(bns)
                if (check_isin[0] == 0) & (check_isin[1] == 0):
                    count_tsuc_and_test_isout += 1
                elif (check_isin[0] == 1) & (check_isin[1] == 0):
                    count_tsuc_and_test_isin += 1
                    interim_count = 0
                    for idx002 in range(t-1):
                        if (bns[idx002][0] == testbns[0][0]) & (bns[idx002][1] == testbns[0][1]):
                            interim_count = 1
                    if interim_count == 0:
                        count_test_isin_and_bad += 1
                        if dp == 1:
                            dup_count_test_isin_and_bad += 1
                    else:
                        count_test_isin_and_good += 1
                        if dp == 1:
                            dup_count_test_isin_and_good += 1
                elif (check_isin[0] == 0) & (check_isin[1] == 1):
                    count_tsuc_and_test_isin += 1
                    interim_count = 0
                    for idx002 in range(t-1):
                        if (bns[t-1+idx002][0] == testbns[1][0]) & (bns[t-1+idx002][1] == testbns[1][1]):
                            interim_count = 1
                    if interim_count == 0:
                        count_test_isin_and_bad += 1
                        if dp == 1:
                            dup_count_test_isin_and_bad += 1
                    else:
                        count_test_isin_and_good += 1
                        if dp == 1:
                            dup_count_test_isin_and_good += 1
                elif (check_isin[0] == 1) & (check_isin[1] == 1):
                    count_tsuc_and_test_isin += 1
                    interim_count = 0
                    interim_count2 = 0
                    for idx002 in range(t-1):
                        if (bns[idx002][0] == testbns[0][0]) & (bns[idx002][1] == testbns[0][1]):
                            interim_count = 1
                    for idx002 in range(t-1):
                        if (bns[t-1+idx002][0] == testbns[1][0]) & (bns[t-1+idx002][1] == testbns[1][1]):
                            interim_count2 = 1
                    trim_count = interim_count + interim_count2
                    if trim_count == 0:
                        count_test_isin_and_badx2 += 1
                        if dp ==1:
                            dup_count_test_isin_and_badx2 += 1
                    elif trim_count == 1:
                        count_test_isin_and_neutral += 1
                        if dp ==1:
                            dup_count_test_isin_and_neutral += 1
                    else:
                        count_test_isin_and_goodx2 += 1
                        if dp ==1:
                            dup_count_test_isin_and_goodx2 += 1
    print('FINAL *********************************************************')
    print(f'count={count} count_con={count_con} count_train_success={count_train_success}')
    print(f'count_tsuc_and_test_isin={count_tsuc_and_test_isin} count_tsuc_and_test_isout={count_tsuc_and_test_isout}')
    print(f'count_test_isin_and_good={count_test_isin_and_good} count_test_isin_and_bad={count_test_isin_and_bad} count_test_isin_and_neutral={count_test_isin_and_neutral}')
    print(f'count_test_isin_and_goodx2={count_test_isin_and_goodx2} count_test_isin_and_badx2={count_test_isin_and_badx2}')
    print(f'dup_count_test_isin_and_good={dup_count_test_isin_and_good} dup_count_test_isin_and_bad={dup_count_test_isin_and_bad} dup_count_test_isin_and_neutral={dup_count_test_isin_and_neutral}')
    print(f'dup_count_test_isin_and_goodx2={dup_count_test_isin_and_goodx2} dup_count_test_isin_and_badx2={dup_count_test_isin_and_badx2}')
    print('____________________________________________')
    print('____________________________________________')

In [8]:
numba.set_num_threads(25)

In [9]:
start = time.perf_counter()

full(perms, terms, maxvalue, Arecplays)

finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')

count=1 count_con=0 count_train_success=0
count_tsuc_and_test_isin=0 count_tsuc_and_test_isout=0
count_test_isin_and_good=0 count_test_isin_and_bad=0 count_test_isin_and_neutral=0
count_test_isin_and_goodx2=0 count_test_isin_and_badx2=0
____________________________________________
count=78048685 count_con=34442484 count_train_success=43606200
count_tsuc_and_test_isin=19320339 count_tsuc_and_test_isout=24285861
count_test_isin_and_good=11501710 count_test_isin_and_bad=5611378 count_test_isin_and_neutral=2001560
count_test_isin_and_goodx2=103690 count_test_isin_and_badx2=102001
____________________________________________
count=78109736 count_con=34503535 count_train_success=43606200
count_tsuc_and_test_isin=19320339 count_tsuc_and_test_isout=24285861
count_test_isin_and_good=11501710 count_test_isin_and_bad=5611378 count_test_isin_and_neutral=2001560
count_test_isin_and_goodx2=103690 count_test_isin_and_badx2=102001
____________________________________________
count=108326273 count_con=

In [ ]:
# started at 10:30 laptop time
# finished around 11:30 laptop time

9765625

1878800

1274180